In [ ]:

# Install necessary dependencies
%pip install torch numpy


In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np
from params import DATALOADERS, LAYERS, ACTIVATIONS, LOSSES, OPTIMIZERS


In [ ]:
class DynamicModel(nn.Module):
    def __init__(self):
        super().__init__()
        layers_list = [
            nn.Linear(8, 12),nn.ReLU(),nn.Linear(12, 8),nn.ReLU(),nn.Linear(8, 1),nn.Sigmoid(),
        ]
        self.layers = nn.ModuleList(layers_list)
    def forward(self, x):
        for l in self.layers:
            x = l(x)

        return x


In [ ]:
ds = DATALOADERS["pima"]

X = torch.tensor(ds["X"], dtype=torch.float32)
Y = torch.tensor(ds["y"], dtype=torch.float32).reshape(-1, 1)

model = DynamicModel()
loss_fn = LOSSES["BCE"]
optimizer = OPTIMIZERS["Adam"](
    model.parameters(), 0.001
)

final_loss = -1


In [ ]:
# Training the model

n_epochs = 100
batch_size = 10

for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        batchX = X[i : i + batch_size]
        y_pred = model(batchX)
        y_batch = Y[i : i + batch_size]
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        final_loss = loss

print(f"Final loss: {final_loss}")


In [ ]:
# Evaluating on test set
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

print(f'\nTest Loss: {test_loss/len(test_loader):.4f}')
